In [ ]:
import sys
sys.path.insert(0, '../')

from src.backtest import backtest_strategy, calculate_sharpe_ratio, calculate_max_drawdown, calculate_win_rate
from src.utils import plot_portfolio_performance
from src.data_loader import load_csv_data
import pandas as pd
import numpy as np

In [ ]:
# Load price data
ticker1 = 'KO'
ticker2 = 'PEP'

prices1 = load_csv_data(f'../data_raw/{ticker1}.csv')
prices2 = load_csv_data(f'../data_raw/{ticker2}.csv')

# Use Adjusted Close price
prices = pd.DataFrame({
    ticker1: prices1['Adj Close'],
    ticker2: prices2['Adj Close']
})

print(f"Prices shape: {prices.shape}")

In [ ]:
# Load signals (from model predictions)
# For demonstration, we'll use z-score based signals
features = pd.read_csv(f'../data_processed/features_with_labels_{ticker1}_{ticker2}.csv', index_col=0, parse_dates=True)

# Align signals with prices
signals = features.loc[prices.index, 'label'].fillna(0).astype(int)

print(f"Signals shape: {signals.shape}")
print(f"Signal distribution:")
print(signals.value_counts())

In [ ]:
# Backtest the strategy
backtest_results = backtest_strategy(prices, signals, initial_capital=100000, trading_cost=0.001)

portfolio_value = backtest_results['portfolio_value']
returns = backtest_results['returns']
cumulative_returns = backtest_results['cumulative_returns']

print(f"Portfolio value shape: {portfolio_value.shape}")
print(f"Initial value: ${portfolio_value.iloc[0]:,.2f}")
print(f"Final value: ${portfolio_value.iloc[-1]:,.2f}")

In [ ]:
# Calculate performance metrics
sharpe_ratio = calculate_sharpe_ratio(returns[1:])  # Skip first NaN
max_drawdown = calculate_max_drawdown(cumulative_returns)
win_rate = calculate_win_rate(returns[1:])
total_return = cumulative_returns.iloc[-1] - 1

print("\n=== Strategy Performance Metrics ===")
print(f"Total Return: {total_return:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Max Drawdown: {max_drawdown:.2%}")
print(f"Win Rate: {win_rate:.2%}")

In [ ]:
# Plot portfolio performance
fig = plot_portfolio_performance(portfolio_value, title=f"Portfolio Performance: {ticker1}-{ticker2} Pairs Trading")
plt.tight_layout()
plt.show()